<a href="https://colab.research.google.com/github/ded-mikhalych/TK_SMM_LRs/blob/main/%D0%9B%D0%A06_%D0%A1%D0%B0%D1%84%D0%BE%D0%BD%D0%BE%D0%B2%D0%B0_%D0%9F_%D0%94_%2C_%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B5%D0%BD%D0%BA%D0%BE%D0%B2_%D0%9A_%D0%95_%2C_%D0%9C%D0%B8%D1%85%D0%B0%D0%BB%D1%8C%D1%87%D1%83%D0%BA_%D0%94_%D0%90_%2C_%D0%9E%D0%B4%D0%B8%D0%BD_%D0%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rnd
import itertools

### 6.1 Написать функции кодирования и декодирования для циклического кода (7,4) с порождающим многочленом g(x) = 1 + $x^2$ + $x^3$, исправляющего однократные ошибки и провести исследование этого кодадля одно-, двух- и трёхкратных ошибок.

In [ ]:
def get_G_matr(gen_poly, k):
    matrix = [gen_poly.copy()]
    while len(matrix) != k:
        deleted = gen_poly.pop(-1)
        gen_poly.insert(0, deleted)
        matrix.append(gen_poly.copy())
    return np.array(matrix)

In [ ]:
def decode(gen_poly, w, k, n, t, err_t):
    sx = get_sx(gen_poly, w, n)

    x = np.poly1d([1, 0])

    if err_t != -1:
        error_set = get_error_set(err_t, n)

    for i in range(k):
        sx_i = (sx * x ** i) / gen_poly
        res = [int(elem % 2) for elem in sx_i[1]]
        comparing_res = res[::-1] + [0] * (n - len(res))
        if (err_t == -1 and res.count(1) <= t) or (err_t != -1 and comparing_res in error_set):
            return np.poly1d(res) * x ** (n - i)
    return None

In [ ]:
def get_sx(gen_poly, w, n):
    mod = np.poly1d(w) / gen_poly
    sx = [0 for _ in range(n - len(mod[1]) - 1)] + [elem % 2 for elem in mod[1]]
    return np.poly1d(sx)

In [ ]:
def get_err_word(g_matr, t, err_t = -1):
    k = g_matr.shape[0]
    n = g_matr.shape[1]

    idx = rnd.randint(1, k - 1)
    a = g_matr[idx][:k]
    print(f"Исходное сообщение: {a}")
    gen_poly = np.poly1d(g_matr[0][::-1])
    v = np.poly1d(a[::-1]) * gen_poly
    v = [elem % 2 for elem in v][:n][::-1]
    print(f"Отправленное сообщение: {v}")
    for i in range(t - 1):
        v[i] += 1
        v[i] %= 2
    v[k] += 1
    v[k] %= 2
    print(f"Принятое сообщение с ошибкой: {v}")
    error = decode(gen_poly, v[::-1], k, n, t, err_t)
    if not error is None:
        print(f"Полученный вектор ошибки:\n{error}")
        res = np.poly1d(v[::-1]) + error
        decoded = [int(elem % 2) for elem in res][::-1][:n]
        print(f"Декодированное сообщение: {decoded}")
    else:
        print("Исправление ошибки невозможно")

In [ ]:
gen_poly = [1, 1, 0, 1, 0, 0, 0]
g_matr = get_G_matr(gen_poly, 4)

In [ ]:
t_list = [1, 2, 3]

In [ ]:
for t in t_list:
    print(f"\033[1mИсследование алгоритма декодрования циклического кода (7, 4) при t = {t}\033[0m")
    get_err_word(g_matr, t)
    if (t != t_list[-1]):
        print("\n")

Исследование алгоритма декодрования циклического кода (7, 4) при t = 1
Исходное сообщение: [0 0 0 1]
Отправленное сообщение: [0, 0, 0, 1, 1, 0, 1]
Принятое сообщение с ошибкой: [0, 0, 0, 1, 0, 0, 1]
Полученный вектор ошибки:
   4
1 x
Декодированное сообщение: [0, 0, 0, 1, 1, 0, 1]


Исследование алгоритма декодрования циклического кода (7, 4) при t = 2
Исходное сообщение: [0 1 1 0]
Отправленное сообщение: [0, 1, 0, 1, 1, 1]
Принятое сообщение с ошибкой: [1, 1, 0, 1, 0, 1]
Полученный вектор ошибки:
   8     6
1 x + 1 x
Декодированное сообщение: [1, 1, 0, 1, 0, 1, 1]


Исследование алгоритма декодрования циклического кода (7, 4) при t = 3
Исходное сообщение: [0 0 0 1]
Отправленное сообщение: [0, 0, 0, 1, 1, 0, 1]
Принятое сообщение с ошибкой: [1, 1, 0, 1, 0, 0, 1]
Полученный вектор ошибки:
   9     7
1 x + 1 x
Декодированное сообщение: [1, 1, 0, 1, 0, 0, 1]


### 6.2 Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом g(x) = 1 + $x^3$ + $x^4$ + $x^5$ + $x^6$, исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4. Обратите внимание, что пакет ошибок длины t не означает, что все разряды в пределах этого пакета изменятся (см. лекции).

In [ ]:
def get_error_set(t, n):
    enum = [0, 1] * (t - 1)
    error_lst = list(set(itertools.permutations(enum, t)))
    res_lst = [comb for comb in error_lst if comb[0] != 0]
    res_lst.append(tuple([1 for _ in range(t)]))
    return [list(res) + [0] * (n - len(res)) for res in res_lst]

In [ ]:
gen_poly = [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
g_matr = get_G_matr(gen_poly, 9)

In [ ]:
t_list = [1, 2, 3, 4]

In [ ]:
for t in t_list:
    print(f"\033[1mИсследование алгоритма декодрования циклического кода (15, 9) при t = {t}\033[0m")
    get_err_word(g_matr, t, 3)
    if (t != t_list[-1]):
        print("\n")

Исследование алгоритма декодрования циклического кода (15, 9) при t = 1
Исходное сообщение: [0 0 1 0 0 1 1 1 1]
Отправленное сообщение: [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]
Принятое сообщение с ошибкой: [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1]
Полученный вектор ошибки:
   9
1 x
Декодированное сообщение: [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]


Исследование алгоритма декодрования циклического кода (15, 9) при t = 2
Исходное сообщение: [0 0 0 0 0 0 0 1 0]
Отправленное сообщение: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
Принятое сообщение с ошибкой: [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
Полученный вектор ошибки:
   14     13     12
1 x  + 1 x  + 1 x 
Декодированное сообщение: [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1]


Исследование алгоритма декодрования циклического кода (15, 9) при t = 3
Исходное сообщение: [0 0 0 0 0 0 1 0 0]
Отправленное сообщение: [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
Принятое сообщение с ошибкой: [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,